In [1]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm

%matplotlib inline

In [2]:
files = sorted(glob('../features/*.feather'))
df = pd.concat([pd.read_feather(f) for f in tqdm(files, mininterval=60)], axis=1)

# set card_id as index
df.set_index('card_id', inplace=True)
df.head()

100%|██████████| 413/413 [00:00<00:00, 481.41it/s]


,CLV_ratio,amount_month_ratio_max,amount_month_ratio_mean,amount_month_ratio_min,card_id_cnt_ratio,card_id_cnt_total,card_id_total,category_1_mean,category_2_mean,category_3_mean,...,weekofyear_9,year_0,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,-0.004333,inf,NaN,-inf,0.088462,283.0,283.0,0.000000,2.046154,0.015385,...,0,0,0,0,0,0,0,0,1,0
C_ID_3d0044924f,0.000369,0.098142,-inf,-inf,0.017143,356.0,356.0,0.088571,2.000000,2.200000,...,0,0,0,0,0,0,0,0,1,0
C_ID_d639edf6cd,inf,-inf,-inf,-inf,0.023256,44.0,44.0,0.000000,9.627907,0.000000,...,0,0,0,0,0,0,0,1,0,0
C_ID_186d6a6901,-inf,NaN,NaN,-inf,0.090909,84.0,84.0,0.298701,6.636364,1.922078,...,0,0,0,0,0,0,0,0,1,0
C_ID_cdbd2c0db2,0.029103,inf,NaN,-inf,0.270677,169.0,169.0,0.168338,6.915831,2.052632,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
corr = df.corr()
corr['target'].to_csv('../share/correlation.csv')